In [ ]:
import numpy as np
import importlib

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from cp2k_spm_tools import cp2k_grid_orbitals, cp2k_ftsts, qe_utils

In [ ]:
lat_param = 4.37 # angstrom

wfn_file = "./examples/polyphenylene_cp2k_scf/PROJ-RESTART.wfn"
xyz_file = "./examples/polyphenylene_cp2k_scf/ppp_12uc-opt.xyz"
cp2k_inp = "./examples/polyphenylene_cp2k_scf/cp2k.inp"
basis_file = "./examples/BASIS_MOLOPT"

In [ ]:
# define global energy limits (eV)
emin = -3.5
emax = 3.5

In [ ]:
cp2k_grid_orb = cp2k_grid_orbitals.Cp2kGridOrbitals()
cp2k_grid_orb.read_cp2k_input(cp2k_inp)
cp2k_grid_orb.read_xyz(xyz_file)
cp2k_grid_orb.ase_atoms.center()
cp2k_grid_orb.read_basis_functions(basis_file)
cp2k_grid_orb.load_restart_wfn_file(wfn_file, emin=emin, emax=emax)

In [ ]:
# define evaluation region (plane)

plane_h = 3.5 # ang

atoms_max_z = np.max(cp2k_grid_orb.ase_atoms.positions[:, 2]) # ang
plane_z = atoms_max_z+plane_h

eval_reg = [None, None, [plane_z, plane_z]]

In [ ]:
cp2k_grid_orb.calc_morbs_in_region(0.10,
                                x_eval_region = eval_reg[0],
                                y_eval_region = eval_reg[1],
                                z_eval_region = eval_reg[2],
                                reserve_extrap = 0.0,
                                pbc = (True, True, False),
                                eval_cutoff = 12.0)

# QE bands (optional)

In [ ]:
qe_scf_xml = "./examples/polyphenylene_qe_bands/scf.xml"
qe_bands_xml = "./examples/polyphenylene_qe_bands/bands.xml"

qe_kpts = None
qe_bands = None
if qe_scf_xml is not None and qe_bands_xml is not None:
    qe_kpts, qe_bands, _ = qe_utils.read_band_data(qe_bands_xml)
    qe_fermi_en = qe_utils.read_scf_data(qe_scf_xml)
    qe_gap_middle = qe_utils.gap_middle(qe_bands[0], qe_fermi_en)
    qe_bands -= qe_gap_middle

# FT-STS

In [ ]:
de = 0.01
fwhm = 0.1

In [ ]:
ftsts = cp2k_ftsts.FTSTS(cp2k_grid_orb)
ftsts.project_orbitals_1d(gauss_pos=0.0, gauss_fwhm=3.0)
borders = ftsts.take_fts(crop_padding=True, crop_edges=1.2, remove_row_avg=True, padding=3.0)
ftsts.make_ftldos(emin, emax, de, fwhm)

In [ ]:
imshow_kwargs = {'aspect': 'auto',
                 'origin': 'lower',
                 #'cmap': 'jet',
                 'cmap': 'gist_ncar',
                }

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 12), gridspec_kw={'width_ratios': [3, 1]})

# left side: LDOS
ax1.imshow(ftsts.ldos.T, extent=ftsts.ldos_extent, vmax=1.0*np.max(ftsts.ldos), **imshow_kwargs)
ax1.axvline(borders[0], color='r')
ax1.axvline(borders[1], color='r')
ax1.set_ylabel("Energy (eV)")
ax1.set_xlabel("x (Å)")

# right side: FT-LDOS
ftldos, extent = ftsts.get_ftldos_bz(2, lat_param) # number of Brilliuin zones
ax2.imshow(ftldos.T, extent=extent, vmax=1.0*np.max(ftldos), **imshow_kwargs)

# add also QE bands
if qe_bands is not None:
    for qe_band in qe_bands[0,]:
        plt.plot(2*qe_kpts[:, 0]*2*np.pi/lat_param, qe_band, '-', color='r', linewidth=2.0)

ax2.set_ylim([emin, emax])
ax2.set_xlabel("2k (Å$^{-1}$)")

plt.show()

# Plot individual orbitals

In [ ]:
# select orbitals wrt to HOMO
index_start = -5
index_end = 6

i_spin = 0

In [ ]:
for i_mo_wrt_homo in range(index_end, index_start-1, -1):
    
    i_mo = cp2k_grid_orb.i_homo_glob[i_spin] + i_mo_wrt_homo
    
    global_i = cp2k_grid_orb.cwf.global_morb_indexes[i_spin][i_mo]
    
    
    print("%d HOMO%+d, E=%.3f eV" % (global_i, i_mo_wrt_homo, cp2k_grid_orb.morb_energies[i_spin][i_mo]))
    morb = (cp2k_grid_orb.morb_grids[i_spin][i_mo, :, :, 0]).astype(np.float64)
    morb_amax = np.max(np.abs(morb))
    
    
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(16, 3))
    plt.subplots_adjust(wspace=0, hspace=0)
    
    ax1.imshow(morb.T, vmin=-morb_amax, vmax=morb_amax,origin='lower', cmap='seismic')
    ax1.set_xticks([])
    ax1.set_yticks([])
    
    ax2.imshow((morb**2).T,origin='lower', cmap='seismic')
    ax2.set_xticks([])
    ax2.set_yticks([])
    
    plt.show()